In [ ]:
#Librerias de Web Scraping
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By

from selenium.webdriver.common.keys import Keys

from bs4 import BeautifulSoup
import requests
import time

In [ ]:
def conexionPaginaWeb():
    driver = webdriver.Chrome(ChromeDriverManager().install())
    #urlUEM
    #url = "https://movielens.org'
    urlMovieLens = "https://www.imdb.com"
    driver.get(urlMovieLens)
    return driver

In [ ]:
def conexionPaginaWeb():
    driver = webdriver.Chrome(ChromeDriverManager().install())
    #urlUEM
    #url = "https://movielens.org'
    urlMovieLens = "https://www.themoviedb.org"
    driver.get(urlMovieLens)
    return driver